In [1]:
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import seaborn as sns

## Read data

In [2]:
# file_name = "../File/FrontendFile/storm-frontend-20200307.txt"
file_name = "./storm-frontend-20200307-mask.txt"

raw_data = pd.read_csv(file_name)

In [3]:
raw_data

,date,time,PID,Level,id,message
0,03/06,03:17:02.880,Thread 5,INFO,18471543-85a3-4707-af7b-ade7801ab25f,process_request : Connection fromIP:2
1,03/06,03:17:02.930,Thread 36,INFO,600c46e5-bea1-4d66-ba00-4819783d705d,Request 'PTP status' from Client IP='IP.154.20...
2,03/06,03:17:02.931,Thread 36,INFO,600c46e5-bea1-4d66-ba00-4819783d705d,Result for request 'PTP status' is 'SRM_REQUES...
3,03/06,03:17:02.995,Thread 30,INFO,ece0cd19-6f59-4a2d-b2bc-8cce75dc3007,Request 'PTG status' from Client IP='IP' Clien...
4,03/06,03:17:02.996,Thread 30,INFO,ece0cd19-6f59-4a2d-b2bc-8cce75dc3007,Result for request 'PTG status' is 'SRM_SUCCESS'
...,...,...,...,...,...,...
9995,03/06,03:20:52.090,Thread 35,INFO,1026667f-61c2-467c-830c-fb963cea06fa,Result for request 'BOL status' is 'SRM_REQUES...
9996,03/06,03:20:52.095,Thread 50,INFO,4fcce4a5-e4c3-4112-94fc-38edfc712358,process_request : Connection fromIP:2
9997,03/06,03:20:52.118,Thread 11,INFO,c6b719a4-d2c1-4fa6-9b79-78c616b4fb8d,Request 'BOL status' from Client IP='IP' Clien...
9998,03/06,03:20:52.120,Thread 11,INFO,c6b719a4-d2c1-4fa6-9b79-78c616b4fb8d,Result for request 'BOL status' is 'SRM_REQUES...


In [4]:
errors = pd.DataFrame()
for i in raw_data.index:
    lista = raw_data.loc[i]
    idx = lista.pop('id')
    temp = pd.DataFrame(dict(lista), index=[idx], copy=True)
    errors = errors.append(temp)

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

...

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errors = errors.append(temp)
C:\Users\Fabri\AppData\Local\Temp\ipykernel_15604\1410790021.py:6: FutureWar

In [5]:
print("Total number of errors:", errors.shape[0])

Total number of errors: 10000


In [6]:
errors['message']

18471543-85a3-4707-af7b-ade7801ab25f                process_request : Connection fromIP:2
600c46e5-bea1-4d66-ba00-4819783d705d    Request 'PTP status' from Client IP='IP.154.20...
600c46e5-bea1-4d66-ba00-4819783d705d    Result for request 'PTP status' is 'SRM_REQUES...
ece0cd19-6f59-4a2d-b2bc-8cce75dc3007    Request 'PTG status' from Client IP='IP' Clien...
ece0cd19-6f59-4a2d-b2bc-8cce75dc3007     Result for request 'PTG status' is 'SRM_SUCCESS'
                                                              ...                        
1026667f-61c2-467c-830c-fb963cea06fa    Result for request 'BOL status' is 'SRM_REQUES...
4fcce4a5-e4c3-4112-94fc-38edfc712358                process_request : Connection fromIP:2
c6b719a4-d2c1-4fa6-9b79-78c616b4fb8d    Request 'BOL status' from Client IP='IP' Clien...
c6b719a4-d2c1-4fa6-9b79-78c616b4fb8d    Result for request 'BOL status' is 'SRM_REQUES...
0dd35adf-2bf2-4b14-bcdf-662a147fd73c    Request 'BOL status' from Client IP='IP' Clien...
Name: mess

## Extract information
Once we read the data, we can focus on the variable message and try to extract meaningful information from it.

### Clearing and tokenizing messages

In [7]:
# Reduce to lowercase and split in tokens
tokens_per_message = [x.lower().split() for x in errors.message]

In [8]:
# Retrieve the set of all tokens used in the error messages
word_set = set()
for mess in tokens_per_message:
    word_set = word_set.union(set(mess))
    
word_set

{'b64f0789-ace8-418b-a0cb-b4c3497b8b88',
 'c556f006-e6cb-4adf-9f71-fb86825db706',
 'd1aeb21b-8ef6-4c55-a9cb-b4b426500333',
 "'703671ec-283e-4cba-9ed3-76249556aa7d'",
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0546._sfo-2._0004.data",
 "'165'",
 '978f6699-fcef-4f85-ae89-12a725f28992',
 'unsupported',
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0433._sfo-7._0003.data_1517905081',
 '46da1075-1fb9-46fb-870b-f94947541c64',
 "'fffb1bc7-55e8-41b0-8fdd-9b269668758a'",
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._008399.job.log.tgz.1.',
 "'b7da8a4e-2614-47a4-b141-5817000e8b0b'",
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._008399.job.log.tgz.1'",
 'srm://storm-fe.cr

In [9]:
print("We have {} error messages, for a total of {} unique tokens adopted.".format(
    len(tokens_per_message), len(word_set)))

We have 10000 error messages, for a total of 1874 unique tokens adopted.


In [10]:
# Setup a dictionary with frequency of all tokens per each message (initialized to 0)
word_dict = [dict.fromkeys(word_set, 0) for i in range(len(tokens_per_message))]
print("Number of tokens:", len(word_dict[0]))

word_dict[0]

Number of tokens: 1874


{'b64f0789-ace8-418b-a0cb-b4c3497b8b88': 0,
 'c556f006-e6cb-4adf-9f71-fb86825db706': 0,
 'd1aeb21b-8ef6-4c55-a9cb-b4b426500333': 0,
 "'703671ec-283e-4cba-9ed3-76249556aa7d'": 0,
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0546._sfo-2._0004.data": 0,
 "'165'": 0,
 '978f6699-fcef-4f85-ae89-12a725f28992': 0,
 'unsupported': 0,
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0433._sfo-7._0003.data_1517905081': 0,
 '46da1075-1fb9-46fb-870b-f94947541c64': 0,
 "'fffb1bc7-55e8-41b0-8fdd-9b269668758a'": 0,
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._008399.job.log.tgz.1.': 0,
 "'b7da8a4e-2614-47a4-b141-5817000e8b0b'": 0,
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._0083

In [11]:
# Compute raw frequencies of each token per each message
for i in range(len(errors.message)):
    for word in tokens_per_message[i]:
        word_dict[i][word] += 1
        
word_dict[0]

{'b64f0789-ace8-418b-a0cb-b4c3497b8b88': 0,
 'c556f006-e6cb-4adf-9f71-fb86825db706': 0,
 'd1aeb21b-8ef6-4c55-a9cb-b4b426500333': 0,
 "'703671ec-283e-4cba-9ed3-76249556aa7d'": 0,
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0546._sfo-2._0004.data": 0,
 "'165'": 0,
 '978f6699-fcef-4f85-ae89-12a725f28992': 0,
 'unsupported': 0,
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0433._sfo-7._0003.data_1517905081': 0,
 '46da1075-1fb9-46fb-870b-f94947541c64': 0,
 "'fffb1bc7-55e8-41b0-8fdd-9b269668758a'": 0,
 'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._008399.job.log.tgz.1.': 0,
 "'b7da8a4e-2614-47a4-b141-5817000e8b0b'": 0,
 "'srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatadisk/rucio/mc15_13tev/be/6e/log.20604254._0083

In [12]:
print("Dictionary dimensions:\n", pd.DataFrame(word_dict).shape)

# Visualization
pd.DataFrame(word_dict)

Dictionary dimensions:
 (10000, 1874)


,b64f0789-ace8-418b-a0cb-b4c3497b8b88,c556f006-e6cb-4adf-9f71-fb86825db706,d1aeb21b-8ef6-4c55-a9cb-b4b426500333,'703671ec-283e-4cba-9ed3-76249556aa7d','srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0546._sfo-2._0004.data,'165',978f6699-fcef-4f85-ae89-12a725f28992,unsupported,srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0433._sfo-7._0003.data_1517905081,46da1075-1fb9-46fb-870b-f94947541c64,...,'64','11','7f6a1b76-2e4c-41ea-afdb-0b93ae93723c',e3df8fcf-84e3-42e2-9d94-41da2340b7da,'524e57cf-035a-4e46-84f3-e8222348e002','e14eb626-1ad7-416a-af93-1bf09c01f095','srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0507._sfo-7._0001.data,816c4fcd-1bf1-44bc-bd8b-00cd6e29764b,srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0639._sfo-3._0001.data,da3b3557-04bd-42ba-8b14-82dd543c6b07
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Compute TF-IDF scores

Once we have messages divided in words and we have computed the raw frequencies of each token in each sentence, then we can proceed and compute the **tf-idf** score for each message.

In [13]:
c = 0
for i, dic in enumerate(tokens_per_message):
    if not len(dic):
        print(i, errors.iloc[i])
        c += 1

In [14]:
print("Warning: there are {} blanck messages which will be excluded from the analysis.".format(c))

In [15]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        try:
            tf[word] = count/sum_nk
        except ZeroDivisionError:
            tf[word] = 0
    return tf

tf = [compute_tf(word_dict[i], tokens_per_message[i]) for i in range(len(tokens_per_message))] #if sum(word_dict[i].values())]
# tf_A = compute_tf(word_dict_A, l_A)
# tf_B = compute_tf(word_dict_B, l_B)
# tf_C = compute_tf(word_dict_C, l_C)

In [16]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = np.log(n / float(v))
    return idf

idf = compute_idf(word_dict)
# idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

In [17]:
len(word_dict), len(tf), len(tf[0]), len(idf)

(10000, 10000, 1874, 1874)

In [18]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf
    
tf_idf =  [compute_tf_idf(tf[i], idf) for i in range(len(tf))]
# tf_idf_A = compute_tf_idf(tf_A, idf)
# tf_idf_B = compute_tf_idf(tf_B, idf)
# tf_idf_C = compute_tf_idf(tf_C, idf)

In [19]:
pd.DataFrame(tf_idf)

,b64f0789-ace8-418b-a0cb-b4c3497b8b88,c556f006-e6cb-4adf-9f71-fb86825db706,d1aeb21b-8ef6-4c55-a9cb-b4b426500333,'703671ec-283e-4cba-9ed3-76249556aa7d','srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0546._sfo-2._0004.data,'165',978f6699-fcef-4f85-ae89-12a725f28992,unsupported,srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0433._sfo-7._0003.data_1517905081,46da1075-1fb9-46fb-870b-f94947541c64,...,'64','11','7f6a1b76-2e4c-41ea-afdb-0b93ae93723c',e3df8fcf-84e3-42e2-9d94-41da2340b7da,'524e57cf-035a-4e46-84f3-e8222348e002','e14eb626-1ad7-416a-af93-1bf09c01f095','srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0507._sfo-7._0001.data,816c4fcd-1bf1-44bc-bd8b-00cd6e29764b,srm://storm-fe.cr.cnaf.infn.it/atlas/atlasdatatape/data17_13tev/raw/other/data17_13tev.00340368.physics_main.daq.raw/data17_13tev.00340368.physics_main.daq.raw._lb0639._sfo-3._0001.data,da3b3557-04bd-42ba-8b14-82dd543c6b07
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Alternative using Scikit-Learn: TfidfVectorizer

In [20]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd


print("Extracting features from the training dataset using a sparse vectorizer")
# t0 = time()
vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.02, stop_words='english',
                             use_idf=True)
# vectorizer = TfidfVectorizer(stop_words='english',
#                              use_idf=True)
X = vectorizer.fit_transform(errors.message)

Extracting features from the training dataset using a sparse vectorizer
CPU times: total: 625 ms
Wall time: 802 ms


In [21]:
vectorizer.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.8,
 'max_features': None,
 'min_df': 0.02,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

## Clustering

In [22]:
# import libraries
from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

# Extract TF-IDF information
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.02, stop_words='english',
                             use_idf=True)
# vectorizer = TfidfVectorizer(stop_words='english',
#                              use_idf=True)
X = vectorizer.fit_transform(errors.message)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()


# Apply LSA for dimensionality reduction to get a lower-dimensional embedding space
print("Performing dimensionality reduction using LSA")
t0 = time()

# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(25)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print()

Extracting features from the training dataset using a sparse vectorizer
done in 0.141174s
n_samples: 10000, n_features: 69

Performing dimensionality reduction using LSA
done in 0.130892s
Explained variance of the SVD step: 99%



Once we have a somewhat convenient subspace for representing the words/tokens, and hence the messages, then we cann proceed and apply the clustering algorithm.

<div class="alert alert-block alert-info">
    <b>Note:</b> In the following, we pretend the variable <i>category</i> as a true label for the clustering results. This is just to show how one could evaluate results of the unsupervised model when target labels are available.
</div>

In [23]:
%%time

# set number of clusters (hyperparameter)
n_clusters = 10

# run K-Means algorithm: 6 clusters
km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=500, n_init=100,
            verbose=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

Clustering sparse data with KMeans(max_iter=500, n_clusters=10, n_init=100, verbose=1)
Initialization complete
Iteration 0, inertia 2117.116232505889
Iteration 1, inertia 1466.055245253509
Iteration 2, inertia 1461.3499141719751
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2425.2604630264577
Iteration 1, inertia 1642.8191480662133
Iteration 2, inertia 1639.9825336851288
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2193.8359980915425
Iteration 1, inertia 1559.8876523052552
Iteration 2, inertia 1544.3265993801886
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2584.6935217904174
Iteration 1, inertia 1674.7031552652002
Iteration 2, inertia 1674.610754472662
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2378.087499240834
Iteration 1, inertia 1655.8395694756405
Iteration 2, inertia 1646.7922894110427
Converged at 

Iteration 2, inertia 1671.9223541868541
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2301.5666862128332
Iteration 1, inertia 1699.4004269798124
Iteration 2, inertia 1699.1368310345792
Iteration 3, inertia 1688.8235345747266
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 2341.0202164436946
Iteration 1, inertia 1597.6023371539854
Iteration 2, inertia 1597.533023116897
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2327.936371440126
Iteration 1, inertia 1589.329835912718
Iteration 2, inertia 1583.1666530973305
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2293.8040655207164
Iteration 1, inertia 1613.5352861161905
Iteration 2, inertia 1613.0468599931266
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2312.0032980697506
Iteration 1, inertia 1680.134647643774
Iteration 2, i

Iteration 0, inertia 2510.994597725958
Iteration 1, inertia 1594.9280791113406
Iteration 2, inertia 1594.4125216550294
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2470.625449894467
Iteration 1, inertia 1719.1120515661569
Iteration 2, inertia 1718.6105177097888
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2658.0851356354137
Iteration 1, inertia 1675.3966771663293
Iteration 2, inertia 1668.763724887319
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2280.263895337228
Iteration 1, inertia 1654.1716811038837
Iteration 2, inertia 1632.8310388940608
Iteration 3, inertia 1632.7175532088477
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 2361.1226859460394
Iteration 1, inertia 1552.2462331703828
Iteration 2, inertia 1550.3081814220693
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, i

## Results
Show graphs and stats here

First thing we can look at are the centroids. 

In [24]:
print("We have {} centroids represented as {}-dimensional points.".format(km.cluster_centers_.shape[0], km.cluster_centers_.shape[1]))
print("Let see the first one for example:")

km.cluster_centers_[0]

We have 10 centroids represented as 25-dimensional points.
Let see the first one for example:


array([ 1.56522188e-01, -4.72230754e-02,  8.03137227e-01, -5.13423835e-01,
       -1.16343088e-01,  7.58878963e-02, -2.57964853e-02, -2.42977612e-02,
       -3.02030872e-04, -4.83139243e-03, -7.47896214e-03,  1.54573856e-02,
        4.24514622e-03, -6.86070088e-03,  3.62251394e-03, -1.37951717e-02,
       -2.70745601e-03, -4.61766266e-03,  8.06001284e-04,  3.56671370e-03,
       -3.05509032e-03, -2.51308428e-03,  4.14951871e-03,  5.70787386e-03,
        1.96628601e-03])

Observations labels, instead, can be accessed through the attribute **labels_** of the clustering object:

In [25]:
from collections import Counter, defaultdict

# print the numerosity of each cluster
print(Counter(km.labels_))

Counter({7: 2121, 2: 1784, 0: 1355, 1: 1168, 8: 816, 6: 815, 4: 769, 5: 668, 3: 283, 9: 221})


Now we can look for example to the messages that fall into the same class:

In [26]:
errors["kmean_labels"] = km.labels_

for msg in errors.message[errors.kmean_labels==8]:
    print(msg)

Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for reque

Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for request 'Ls' is 'SRM_SUCCESS'
Result for reque

### Check the true label (if available)

In [27]:
# extract true clusters
labels = errors.PID

# evaluate results comparing with labels
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

Homogeneity: 0.007
Completeness: 0.014
V-measure: 0.009
Adjusted Rand-Index: 0.000
Silhouette Coefficient: 0.616


In [28]:
print("There are {} unique categories for error messages.".format(len(np.unique(labels))))

There are 64 unique categories for error messages.


## Conclusions and Next Steps
Summarize findings here